# LangChain: Agents

Agents use an LLM to determine which actions to take and in what order. 

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

## Built-in LangChain tools

In [ ]:
# !pip install -U wikipedia

In [8]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [9]:
llm = ChatOpenAI(temperature=0)

In [10]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [11]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [12]:
agent.run("What is the 25% of 300?")

Error in on_chain_start callback: 'name'


Thought: We need to calculate 25% of 300, which involves multiplication and division.

Action:
```
{
  "action": "Calculator",
  "action_input": "300*0.25"
}
```


Observation: Answer: 75.0
Thought:We have the answer to the question.

Final Answer: 75.0

> Finished chain.


'75.0'

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
agent.run(question)

In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [2]:
llm = ChatOpenAI(temperature=0)

In [ ]:
# !pip install google-search-results

In [17]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [18]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [7]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

Error in on_chain_start callback: 'name'


I need to find out Leo DiCaprio's girlfriend's name and her age.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Leonardo DiCaprio has split from girlfriend Camila Morrone. Getty. Taking a look at his dating history, Leo has always had a preference for ...
Thought:I need to find out Camila Morrone's age.
Action: Search
Action Input: "Camila Morrone age"
Observation: 25 years
Thought:I need to calculate her age raised to the 0.43 power.
Action: Calculator
Action Input: 25^(0.43)
Observation: Answer: 3.991298452658078
Thought:I now know the final answer.
Final Answer: 3.991298452658078

> Finished chain.


'3.991298452658078'

In [33]:
result = agent.run("今天上海的天气如何？")

Error in on_chain_start callback: 'name'


Thought: I need to use the Search tool to find the weather in Shanghai today.
Action:
```
{
  "action": "Search",
  "action_input": "Shanghai weather today"
}
```

Observation: TOMORROW'S WEATHER FORECAST. 6/14. 88°/ 72°. RealFeel® Sun 94 ...
Thought:I need to refine my search query to get the weather for today in Shanghai.
Action:
```
{
  "action": "Search",
  "action_input": "Shanghai weather today"
}
```


Observation: TOMORROW'S WEATHER FORECAST. 6/14. 88°/ 72°. RealFeel® Sun 94 ...
Thought:I need to try a different search engine to get the weather for today in Shanghai.
Action:
```
{
  "action": "Search",
  "action_input": "Shanghai weather today Baidu"
}
```


Observation: Sun. Today · 93 °F · 4 mph. - ; Mon. Tomorrow · 94 °F · 5 mph. - ; Tue. 6-13 · 89 °F · 3 mph. 0-0.4".
Thought:I have found that the temperature in Shanghai today is 93°F and the wind speed is 4 mph.
Final Answer: The temperature in Shanghai today is 93°F and the wind speed is 4 mph.

> Finished chain.


In [34]:
result

'The temperature in Shanghai today is 93°F and the wind speed is 4 mph.'

## Python Agent

In [21]:
from langchain.agents.agent_toolkits import create_python_agent

In [22]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [23]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [24]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

Error in on_chain_start callback: 'name'


I can use the sorted() function to sort the list of customers by last name and then first name. I will need to provide a key function to sorted() that returns a tuple of the last name and first name in that order.
Action: Python REPL
Action Input:
```
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
```
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:The customers are now sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

## Define your own tool

the Tool consists of several components:

- name (str), is required and must be unique within a set of tools provided to an agent

- description (str), is optional but recommended, as it is used by an agent to determine tool use

- return_direct (bool), defaults to False

- args_schema (Pydantic BaseModel), is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.

In [25]:
# !pip install DateTime

In [26]:
from langchain.agents import tool
from datetime import date

In [27]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [28]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [29]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")

Error in on_chain_start callback: 'name'


Thought: We can use the `time` tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-06-13
Thought:The answer to the question "what's the date today?" is 2023-06-13.
Final Answer: 2023-06-13.

> Finished chain.


In [35]:
from langchain.agents import load_huggingface_tool

tool = load_huggingface_tool("lysandre/hf-model-downloads")

print(f"{tool.name}: {tool.description}")

A new version of the following files was downloaded from https://huggingface.co/spaces/lysandre/hf-model-downloads:
- model_downloads.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model_download_counter: This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub. It takes the name of the category (such as text-classification, depth-estimation, etc), and returns the name of the checkpoint


In [36]:
tool.run("text-classification")

'facebook/bart-large-mnli'

### Define customer tools

In [38]:
# Import things that are needed generically
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [39]:
llm = ChatOpenAI(temperature=0)

In [40]:
# Load the tool configs that are needed.
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name = "Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

/opt/homebrew/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [41]:
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
    question: str = Field()
        

tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

In [42]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [43]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

Error in on_chain_start callback: 'name'


I need to find out Leo DiCaprio's girlfriend's name and her age
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Leonardo DiCaprio has split from girlfriend Camila Morrone. Getty. Taking a look at his dating history, Leo has always had a preference for ...
Thought:I still need to find out her age
Action: Search
Action Input: "Camila Morrone age"
Observation: 25 years
Thought:

Error in on_chain_start callback: 'name'


I need to calculate her age raised to the 0.43 power
Action: Calculator
Action Input: 25^0.4325^0.43```text
25**0.43
```
...numexpr.evaluate("25**0.43")...

Answer: 3.991298452658078
> Finished chain.

Observation: Answer: 3.991298452658078
Thought:I now know the final answer
Final Answer: 3.991298452658078

> Finished chain.


'3.991298452658078'